# IBM Data Science Capstone Project  
## Lab 5: Interactive Visual Analytics with Folium

### Project Title: SpaceX Falcon 9 First Stage Landing Prediction  
**Assignment**: Interactive Visual Analytics with Folium


## Objective

In this lab, we aim to explore how geographic factors may influence Falcon 9 first stage landing success rates. This includes:

- Visualizing the location of launch sites
- Annotating successful and failed launches
- Measuring distances from launch sites to nearby features (coastlines, railways, etc.)

This helps uncover geographic patterns that may influence rocket landing success.


## Setup Environment

In [ ]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

## Load Dataset

In [ ]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

## Clean and filter

In [ ]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()[['Launch Site', 'Lat', 'Long']]

## 🗺️ Task 1: Mark All Launch Sites

In [ ]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circles and markers for each site
for index, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    folium.Circle(coord, radius=1000, color='#000000', fill=True).add_child(
        folium.Popup(row['Launch Site'])).add_to(site_map)
    folium.map.Marker(
        coord,
        icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{row["Launch Site"]}</b></div>')
    ).add_to(site_map)

## 🟢🔴 Task 2: Mark Success/Failed Launches

In [ ]:
marker_cluster = MarkerCluster()

# Assign colors
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

for index, record in spacex_df.iterrows():
    coord = [record['Lat'], record['Long']]
    marker = folium.Marker(
        location=coord,
        icon=folium.Icon(color=record['marker_color']),
        popup=f"{record['Launch Site']} - {'Success' if record['class']==1 else 'Fail'}"
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)

## Add MousePosition

In [ ]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

## 📏 Task 3: Distance to Proximities

In [ ]:
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

    # Example: CCAFS LC-40 to nearest coast

    launch_site = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS LC-40'].iloc[0]
site_lat, site_lon = launch_site['Lat'], launch_site['Long']
coast_lat, coast_lon = 28.56367, -80.57163  # Retrieved manually

distance = calculate_distance(site_lat, site_lon, coast_lat, coast_lon)

folium.Marker(
    [coast_lat, coast_lon],
    icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>')
).add_to(site_map)

folium.PolyLine(
    locations=[[site_lat, site_lon], [coast_lat, coast_lon]], weight=2
).add_to(site_map)


####  Optional: Repeat for other features (railway, highway, city)

### # Show the final map
site_map


## ✅ Summary
Visualized launch sites and outcomes interactively

Calculated geographic proximities

Identified patterns between location and launch success



### Complete the Lab 5: Interactive Visual Analytics with Folium
Md. Anwar Hossain